In [1]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
from data_sources.clob import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

In [3]:
# Constants
CONNECTOR_NAME = "binance"
INTERVAL = "15m"
DAYS = 20


# Features configuration
VOLATILITY_WINDOW = 100
TREND_SHORT_WINDOW = 20
TREND_LONG_WINDOW = 100
VOLUME_SHORT_WINDOW = 20
VOLUME_LONG_WINDOW = 100

# Download data
- Get trading rules
- Get candles for the last x days

In [4]:
import asyncio

trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset("FDUSD")\
    .filter_by_min_notional_size(Decimal("5"))\
    .get_all_trading_pairs()

2024-08-21 12:21:37,500 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x16993a2c0>
2024-08-21 12:21:37,501 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1183d3880>, 4752.11036175)]']
connector: <aiohttp.connector.TCPConnector object at 0x16993a2f0>


In [5]:
tasks = [clob.get_candles_last_days(
    connector_name=CONNECTOR_NAME,
    trading_pair=trading_pair,
    interval=INTERVAL,
    days=DAYS,
) for trading_pair in trading_pairs]

candles = await asyncio.gather(*tasks)
candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}

2024-08-21 12:21:39,425 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x16b1e3c70>
2024-08-21 12:21:39,425 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x16adbfa60>, 4754.021309708)]']
connector: <aiohttp.connector.TCPConnector object at 0x16b1e3ca0>
2024-08-21 12:21:39,616 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x16b0ec4f0>
2024-08-21 12:21:39,616 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x16b1ea620>, 4754.2216855)]']
connector: <aiohttp.connector.TCPConnector object at 0x16b0ec520>
2024-08-21 12:21:39,713 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x16b25e5f0>
2024-08-21 12:21:39,714 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x16aea6e

In [6]:
from features.candles.trend import TrendConfig
from features.candles.volatility import VolatilityConfig
from features.candles.volume import VolumeConfig
from research_notebooks.dneitor.utils import generate_report

report = generate_report(
    candles=candles,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    trend_config=TrendConfig(short_window=TREND_SHORT_WINDOW, long_window=TREND_LONG_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_SHORT_WINDOW, long_window=VOLUME_LONG_WINDOW))
report

,trading_pair,mean_volatility,mean_natr,mean_bb_width,latest_trend,average_volume_per_hour,normalized_score
0,RENDER-FDUSD,0.00744348,0.00817228,9.85996207,0.2509625,11395.76763475,0.00017496
1,IOTA-FDUSD,0.00459254,0.00258779,6.32422338,0.79377139,281.5104717,0.00000277
2,OP-FDUSD,0.00610204,0.00647417,8.82410967,-0.13925753,13090.95211027,0.00017981
3,WIF-FDUSD,0.00884504,0.01187004,13.33269495,-0.05239287,99739.8550075,0.00207118
4,XAI-FDUSD,0.00703882,0.00607857,9.37282473,-0.22849552,2331.52647662,0.000034
...,...,...,...,...,...,...,...
88,PENDLE-FDUSD,0.00831053,0.00833768,13.49763733,0.29976752,12024.0573999,0.0002525
89,PEOPLE-FDUSD,0.00764307,0.0062158,10.4981899,0.95216076,5570.67393097,0.00009098
90,SEI-FDUSD,0.00674111,0.00875172,9.94450872,0.03368901,19776.98236186,0.00030631
91,INJ-FDUSD,0.00706252,0.00675461,10.05759259,-0.07225744,9819.02655612,0.00015368


In [7]:
from research_notebooks.dneitor.utils import filter_top_markets

TOP_X_MARKETS = 5  # Number of top markets to select
MIN_VOLUME_USD = 2000  # Minimum volume in USD
MIN_NATR = 0.008  # Minimum ATR
TREND_THRESHOLD = -0.5  # Trend threshold

top_markets = filter_top_markets(report_df=report, top_x=TOP_X_MARKETS, min_volume_usd=MIN_VOLUME_USD, min_natr=MIN_NATR, trend_threshold=TREND_THRESHOLD)
top_markets

,trading_pair,mean_volatility,mean_natr,mean_bb_width,latest_trend,average_volume_per_hour,normalized_score
67,SOL-FDUSD,0.00586641,0.00878255,8.04324601,-0.00587164,17761677.5246158,0.22268711
37,PEPE-FDUSD,0.00747568,0.01078449,10.82986292,-0.13232489,183615.93440093,0.00309847
3,WIF-FDUSD,0.00884504,0.01187004,13.33269495,-0.05239287,99739.8550075,0.00207118
59,NOT-FDUSD,0.00714142,0.00971149,10.86277941,-0.01533987,90489.69368465,0.00153101
73,BANANA-FDUSD,0.01101322,0.01349592,14.65625442,0.12893465,30824.16017784,0.00070372


In [8]:
from research_notebooks.dneitor.utils import generate_config, dump_dict_to_yaml


TOTAL_AMOUNT = 3000  # General total amount for all markets
MIN_AMOUNT_PER_MARKET = 200  # Minimum amount per market
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_OPEN_ORDERS = 1  # Input max open orders for each market

TAKE_PROFIT_MULTIPLIER = 0.3  # Multiplier for take profit based on NATR
AMOUNTS_QUOTE_PCT = [0.1, 0.1, 0.1, 0.1, 0.2, 0.2]  # Weights for each cluster

strategy_config = generate_config(
    id="dneitor-binance_0.4",
    connector_name=CONNECTOR_NAME,
    candles=candles,
    top_markets=top_markets,
    total_amount=TOTAL_AMOUNT,
    amounts_quote_pct=AMOUNTS_QUOTE_PCT,
    activation_bounds=ACTIVATION_BOUNDS,
    take_profit_multiplier=TAKE_PROFIT_MULTIPLIER,
    max_open_orders=MAX_OPEN_ORDERS,
    min_amount_per_market=MIN_AMOUNT_PER_MARKET,
)
dump_dict_to_yaml("configs/", strategy_config)